In [1]:
import pandas as pd

In [2]:
df = pd.read_csv(f'./astronauts.csv')

In [3]:
df.columns

Index(['s.no/code given', 'Astronaut name ', 'country', 'company space agency',
       'missions', 'days in space', 'spacewalks', 'days spacewalking',
       'acheivement 1', 'acheivement 2', 'acheivement 3', 'acheivement 4',
       'still in space'],
      dtype='object')

In [4]:
df.head()

,s.no/code given,Astronaut name,country,company space agency,missions,days in space,spacewalks,days spacewalking,acheivement 1,acheivement 2,acheivement 3,acheivement 4,still in space
0,612,Cameron Bess,United States of America,Blue Origin,1,<.1,-,-,Crossed Kármán Line,NaN,NaN,NaN,NaN
1,611,Lane Bess,United States of America,Blue Origin,1,<.1,-,-,Crossed Kármán Line,NaN,NaN,NaN,NaN
2,610,Evan Dick,United States of America,Blue Origin,1,<.1,-,-,Crossed Kármán Line,NaN,NaN,NaN,NaN
3,609,Dylan Taylor,United States of America,Blue Origin,1,<.1,-,-,Crossed Kármán Line,NaN,NaN,NaN,NaN
4,608,Michael Strahan,United States of America,Blue Origin,1,<.1,-,-,Crossed Kármán Line,NaN,NaN,NaN,NaN


In [5]:
df.describe()

,s.no/code given,Astronaut name,country,company space agency,missions,days in space,spacewalks,days spacewalking,acheivement 1,acheivement 2,acheivement 3,acheivement 4,still in space
count,942,952,952,629,952,621,621,621,734,403,215,331,10
unique,942,951,42,22,8,370,12,30,133,43,21,132,1
top,612,Unknown Dog,United States of America,NASA,1,<.1,-,-,Crossed Kármán Line,Crossed Kármán Line,Space Resident,Dog,yes
freq,1,2,523,364,540,25,379,379,325,266,146,47,10


In [6]:
df['missions'].unique()

array(['1', '2', '3', '4', '5', '6', '7', '-'], dtype=object)

In [7]:
df = df[df.missions != '-']

In [8]:
df = df[(df['days in space'] != '<.1') & (df['days in space'] != '-')]

In [9]:
df['missions']=df['missions'].astype(int)

In [10]:
df = df[df['days in space'].notna()]

In [11]:
df['days in space']=df['days in space'].astype(float).astype(int)

In [12]:
# for the second part of the point 1 ""% of astronauts in space by country."
#I will asume that spacewalks is the same as astronauts in space.
#To deal with nan values I will fill them with 0
df["spacewalks"] = df["spacewalks"].fillna(0)
df["spacewalks"] = df["spacewalks"].replace(['-'], 0)

In [13]:
df['spacewalks']=df['spacewalks'].astype(int)

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 577 entries, 6 to 871
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   s.no/code given       577 non-null    object
 1   Astronaut name        577 non-null    object
 2   country               577 non-null    object
 3   company space agency  577 non-null    object
 4   missions              577 non-null    int32 
 5   days in space         577 non-null    int32 
 6   spacewalks            577 non-null    int32 
 7   days spacewalking     577 non-null    object
 8   acheivement 1         577 non-null    object
 9   acheivement 2         399 non-null    object
 10  acheivement 3         214 non-null    object
 11  acheivement 4         0 non-null      object
 12  still in space        10 non-null     object
dtypes: int32(3), object(10)
memory usage: 56.3+ KB


In [15]:
df.to_csv('./astronauts_wrangling.csv')

# Group by

In [16]:
import numpy as np

In [17]:
##obtener promedio de misiones y dias en el espacio por pais
df_fixed = df[['country', 'missions', 'days in space', 'spacewalks']]
#df_grouped = df_fixed.groupby(['country'])['missions', 'days in space'].mean()

df_grouped = df_fixed.groupby(['country']).agg({'missions': np.mean,
                                                'days in space': np.mean,
                                                'spacewalks': np.sum})


In [18]:
df_grouped.tail()

,missions,days in space,spacewalks
country,,,
USSR,2.225352,148.563380,116
Ukraine,1.000000,15.000000,0
United Arab Emirates,1.000000,7.000000,0
United States of America,2.522989,67.859195,524
Vietnam,1.000000,10.000000,0


In [19]:
total_spacewalks = df_grouped['spacewalks'].sum()
total_spacewalks

867

In [20]:
df_grouped['percentage_astronauts'] = (df_grouped['spacewalks']/total_spacewalks)*100
df_grouped.head()

,missions,days in space,spacewalks,percentage_astronauts
country,,,,
Afghanistan,1.0,8.0,0,0.000000
Austria,1.0,7.0,0,0.000000
Belgium,1.5,103.0,0,0.000000
Brazil,1.0,9.0,0,0.000000
Bulgaria,1.0,5.5,2,0.230681


In [21]:
#get missions by country by company
df_fixed2 = df[['country', 'missions', 'company space agency']]
df_g = df_fixed2.groupby(['country', 'company space agency']).agg({'missions': np.sum}).reset_index()
df_g = df_g.sort_values('missions', ascending=False)

In [22]:
df_g.head()

,country,company space agency,missions
48,United States of America,NASA,871
45,USSR,CCCP,158
34,Russia,Roscosmos,130
7,China,CMS,20
25,Japan,NASDA,18


In [23]:
df_gfinal=df_g.groupby(['country', 'company space agency'])['missions'].max().reset_index()

In [40]:
df_gf=df_gfinal.sort_values('missions', ascending=False).drop_duplicates('country').sort_index()
df_gf.head()

,country,company space agency,missions
0,Afghanistan,CCCP,1
1,Austria,Roscosmos,1
2,Belgium,ESA,3
3,Brazil,NASA,1
4,Bulgaria,Roscosmos,2


In [42]:
df_f1 = df_grouped.merge(df_gf, left_on='country', right_on='country')
df_f1.head()

,country,missions_x,days in space,spacewalks,percentage_astronauts,company space agency,missions_y
0,Afghanistan,1.0,8.0,0,0.000000,CCCP,1
1,Austria,1.0,7.0,0,0.000000,Roscosmos,1
2,Belgium,1.5,103.0,0,0.000000,ESA,3
3,Brazil,1.0,9.0,0,0.000000,NASA,1
4,Bulgaria,1.0,5.5,2,0.230681,Roscosmos,2


In [24]:
df2 = pd.read_csv(f'./astronauts_wrangling.csv')

In [25]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 577 entries, 0 to 576
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Unnamed: 0            577 non-null    int64  
 1   s.no/code given       577 non-null    int64  
 2   Astronaut name        577 non-null    object 
 3   country               577 non-null    object 
 4   company space agency  577 non-null    object 
 5   missions              577 non-null    int64  
 6   days in space         577 non-null    int64  
 7   spacewalks            577 non-null    int64  
 8   days spacewalking     577 non-null    object 
 9   acheivement 1         577 non-null    object 
 10  acheivement 2         399 non-null    object 
 11  acheivement 3         214 non-null    object 
 12  acheivement 4         0 non-null      float64
 13  still in space        10 non-null     object 
dtypes: float64(1), int64(5), object(8)
memory usage: 63.2+ KB


In [47]:
df_fixed3 = df[['country', 'acheivement 1']]
df_fixed3['count1'] = 1

df_g2 = df_fixed3.groupby(['country', 'acheivement 1'])['count1'].sum().reset_index()

C:\Users\diego\AppData\Local\Temp\ipykernel_7340\1816565514.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_fixed3['count1'] = 1


In [48]:
df_g2.head()

,country,acheivement 1,count1
0,Afghanistan,Crossed Kármán Line,1
1,Austria,Crossed Kármán Line,1
2,Belgium,Crossed Kármán Line,1
3,Belgium,ISS Visitor,1
4,Brazil,ISS Visitor,1


In [50]:
df_gf2=df_g2.sort_values('count1').drop_duplicates('country').sort_index()
df_gf2.head()

,country,acheivement 1,count1
0,Afghanistan,Crossed Kármán Line,1
1,Austria,Crossed Kármán Line,1
3,Belgium,ISS Visitor,1
4,Brazil,ISS Visitor,1
5,Bulgaria,Crossed Kármán Line,2


In [51]:
df_f2 = df_f1.merge(df_gf2, left_on='country', right_on='country')
df_f2.head()

,country,missions_x,days in space,spacewalks,percentage_astronauts,company space agency,missions_y,acheivement 1,count1
0,Afghanistan,1.0,8.0,0,0.000000,CCCP,1,Crossed Kármán Line,1
1,Austria,1.0,7.0,0,0.000000,Roscosmos,1,Crossed Kármán Line,1
2,Belgium,1.5,103.0,0,0.000000,ESA,3,ISS Visitor,1
3,Brazil,1.0,9.0,0,0.000000,NASA,1,ISS Visitor,1
4,Bulgaria,1.0,5.5,2,0.230681,Roscosmos,2,Crossed Kármán Line,2


In [54]:
df_ff= df_f2.drop(['missions_y', 'count1'], axis=1)
df_ff

,country,missions_x,days in space,spacewalks,percentage_astronauts,company space agency,acheivement 1
0,Afghanistan,1.000000,8.000000,0,0.000000,CCCP,Crossed Kármán Line
1,Austria,1.000000,7.000000,0,0.000000,Roscosmos,Crossed Kármán Line
2,Belgium,1.500000,103.000000,0,0.000000,ESA,ISS Visitor
3,Brazil,1.000000,9.000000,0,0.000000,NASA,ISS Visitor
4,Bulgaria,1.000000,5.500000,2,0.230681,Roscosmos,Crossed Kármán Line
5,Canada,1.800000,70.400000,7,0.807382,CSA,Crossed Kármán Line
6,China,1.538462,67.230769,10,1.153403,CMS,Space Resident
7,Cuba,1.000000,7.000000,0,0.000000,CCCP,Crossed Kármán Line
8,Czechoslovakia,1.000000,7.000000,0,0.000000,CCCP,Crossed Kármán Line
9,Denmark,1.000000,9.000000,0,0.000000,ESA,ISS Visitor
